## Matriz confusion y precision
La matriz confusion es una matriz de doble entrada para comparar dos particiones. Cada elemento representa la coincidencia entre ambas particiones: cuenta pares de nodos repetidos (si pertenecen o no a un mismo cluster).

In [1]:
import networkx as nx
import matplotlib.pylab as plt
%matplotlib inline
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

In [2]:
carpeta = os.getcwd()+'/'

G = nx.read_gml(carpeta+'dolphins.gml')

gender = pd.read_csv(carpeta+'dolphinsGender.txt', sep='\t', header=None)
delfines = gender[0] # nombres de delfines (nodos)
genero = gender[1]   # genero de los delfines en el mismo orden que los nodos
n = len(delfines) # cantidad de delfines

# A cada uno de los nodos ya existentes en el grafo, se le agrega el genero
for d,g in zip(delfines,genero):
    G.add_node(d, gender=g)

nombres = ['fast_greedy','edge_betweenness','louvain']

for metodo in nombres:
    data = pd.read_csv('./comparacion/labels_'+metodo+'.txt', sep='\t')
    dolphins = data['Nombre'] # nombres de delfines
    comu = data['Comunidad'] # comunidad a la que pertenecen
    
    for d,c in zip(dolphins,comu):
        G.node[d][metodo]=c
        
# El grafo G tiene los nodos con atributos genero, fast_greedy, edge_betweenness, y louvain
# (los ultimos tres atributos son el numero de comunidad en cada uno de esos metodos)

In [51]:
#Lo hacemos para fast-greedy y edge-betweeness, para  fast-greedy y Louvain y para edge-betweeness y Louvain
precision=[]


nombres=['fast_greedy','edge_betweenness','louvain','fast_greedy']
for i in range(len(nombres)-1):

    vector1 = []
    vector2 = []


    metodo1 = nombres[i] 
    metodo2 = nombres[i+1]

    for i in range(n):
        for j in range(i,n):
            if G.node[delfines[i]][metodo1] == G.node[delfines[j]][metodo1]:
                vector1.append(1)
            else:
                vector1.append(0)

    for i in range(n):
        for j in range(i,n):
            if G.node[delfines[i]][metodo2] == G.node[delfines[j]][metodo2]:
                vector2.append(1)
            else:
                vector2.append(0)


    cnf_matrix=confusion_matrix(vector1,vector2) #en el primer lugar va la "real" y les paso las comunidades
    a=cnf_matrix[0][0]
    b=cnf_matrix[0][1]
    c=cnf_matrix[1][0]
    d=cnf_matrix[1][1]
    
    print(cnf_matrix)
    
    precision_i=(a+b)/(a+b+c+d)
    precision.append(precision_i)

tabla_resumen = pd.DataFrame({"Comunidades":["FG vs EB","EB vs Lou","Lou vs FG"],
                    "Precisión":precision, #es = que modularidades
                   })
tabla_resumen




[[1288  125]
 [ 115  425]]
[[1350   53]
 [ 126  424]]
[[1355  121]
 [  58  419]]


,Comunidades,Precisión
0,FG vs EB,0.723502
1,EB vs Lou,0.718382
2,Lou vs FG,0.755760
